**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
show_env()

You are working on        Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
REPO DIRECTORY (FD_REPO): /data/reddylab/Kuei/repo 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/work 
DATA DIRECTORY (FD_DATA): /data/reddylab/Kuei/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log 
PROJECT REF     (FD_REF): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/references 



## Import data

In [2]:
txt_fdiry = file.path(
    FD_RES, 
    "region_annotation", 
    "fcc_astarr_macs_input_overlap",
    "summary"
)
txt_fname = "matrix.annotation.fcc_peak_call.tsv"
txt_fpath = file.path(txt_fdiry, txt_fname)

dat = read_tsv(txt_fpath, show_col_types = FALSE)

dat_region_annot_fcc = dat
print(dim(dat))
fun_display_table(head(dat, 3))

[1] 100454     15


Chrom,ChromStart,ChromEnd,Region,ASTARR_A,WSTARR_A,LMPRA_A,TMPRA_A,ASTARR_R,WSTARR_R,LMPRA_R,TMPRA_R,CRISPRi-HCRFF,CRISPRi-Growth,CRISPR-E2G
chr1,10038,10405,chr1:10038-10405,0,0,0,0,1,0,0,0,0,0,0
chr1,16025,16338,chr1:16025-16338,0,0,0,0,1,0,0,0,0,0,0
chr1,17288,17689,chr1:17288-17689,0,0,0,0,1,0,0,0,0,0,0


In [3]:
txt_fdiry = file.path(
    FD_RES, 
    "region_nuc", 
    "fcc_astarr_macs",
    "summary"
)
txt_fname = "K562.hg38.ASTARR.macs.KS91.input.rep_all.max_overlaps.q5.tsv"
txt_fpath = file.path(txt_fdiry, txt_fname)

dat = read_tsv(txt_fpath, show_col_types = FALSE)

dat_region_nuc = dat
print(dim(dat))
fun_display_table(head(dat, 3))

[1] 150042      6


Chrom,ChromStart,ChromEnd,Region,pGC,Length
chr1,10038,10405,chr1:10038-10405,0.523161,367
chr1,14282,14614,chr1:14282-14614,0.578313,332
chr1,16025,16338,chr1:16025-16338,0.587859,313


## ASTARR and GC Content

In [4]:
tmp = dat_region_nuc
tmp = tmp  %>% 
    dplyr::mutate(High_GC = as.integer(pGC > 0.60)) %>%
    dplyr::select(Chrom, ChromStart, ChromEnd, High_GC)

dat = dat_region_annot_fcc
dat = dat %>% dplyr::left_join(tmp, by = c("Chrom", "ChromStart", "ChromEnd"))
dat = dat %>% dplyr::mutate(ASTARR_R_GC = ifelse(High_GC == 1, 0, ASTARR_R))

dat_region_annot_fcc_pgc = dat
print(dim(dat))
fun_display_table(head(dat, 3))

[1] 100454     17


Chrom,ChromStart,ChromEnd,Region,ASTARR_A,WSTARR_A,LMPRA_A,TMPRA_A,ASTARR_R,WSTARR_R,LMPRA_R,TMPRA_R,CRISPRi-HCRFF,CRISPRi-Growth,CRISPR-E2G,High_GC,ASTARR_R_GC
chr1,10038,10405,chr1:10038-10405,0,0,0,0,1,0,0,0,0,0,0,0,1
chr1,16025,16338,chr1:16025-16338,0,0,0,0,1,0,0,0,0,0,0,0,1
chr1,17288,17689,chr1:17288-17689,0,0,0,0,1,0,0,0,0,0,0,1,0


## Enhancer vs Silencer

In [5]:
dat = dat_region_annot_fcc
dat = dat %>% dplyr::select(Region, ends_with("_A"))

dat_region_annot_enh = dat
print(dim(dat))
fun_display_table(head(dat, 3))

[1] 100454      5


Region,ASTARR_A,WSTARR_A,LMPRA_A,TMPRA_A
chr1:10038-10405,0,0,0,0
chr1:16025-16338,0,0,0,0
chr1:17288-17689,0,0,0,0


In [6]:
dat = dat_region_annot_fcc
dat = dat %>% dplyr::select(Region, ends_with("_R"))

dat_region_annot_rep = dat
print(dim(dat))
fun_display_table(head(dat, 3))

[1] 100454      5


Region,ASTARR_R,WSTARR_R,LMPRA_R,TMPRA_R
chr1:10038-10405,1,0,0,0
chr1:16025-16338,1,0,0,0
chr1:17288-17689,1,0,0,0


In [7]:
dat = dat_region_annot_fcc_pgc
dat = dat %>% 
    dplyr::select(Region, ASTARR_R_GC, ends_with("_R")) %>%
    dplyr::select(-ASTARR_R)

dat_region_annot_rep_gcfilter = dat
print(dim(dat))
fun_display_table(head(dat, 3))

[1] 100454      5


Region,ASTARR_R_GC,WSTARR_R,LMPRA_R,TMPRA_R
chr1:10038-10405,1,0,0,0
chr1:16025-16338,1,0,0,0
chr1:17288-17689,0,0,0,0


## CRISPRi and CRISPR-E2G

In [8]:
dat = dat_region_annot_fcc

dat = dat %>% 
    dplyr::select(Region, starts_with("CRISPR")) %>% 
    replace(is.na(.), 0)

dat_region_annot_crispr = dat
print(dim(dat))
fun_display_table(head(dat, 3))

[1] 100454      4


Region,CRISPRi-HCRFF,CRISPRi-Growth,CRISPR-E2G
chr1:10038-10405,0,0,0
chr1:16025-16338,0,0,0
chr1:17288-17689,0,0,0


## Count assay for each region

In [9]:
lst = list(
    "Enhance" = dat_region_annot_enh,
    "Repress" = dat_region_annot_rep,
    "Repress_GCFilter" = dat_region_annot_rep_gcfilter,
    "CRISPR"   = dat_region_annot_crispr
)

lst = lapply(lst, function(dat){
    dat = dat %>% tibble::column_to_rownames(var="Region")
    vec = apply(dat, 1, sum)
    
    idx = (vec > 0)
    vec = vec[idx]

    dat = data.frame(
        Region    = names(vec),
        Num_Assay = vec
    )
    return(dat)
})

dat = bind_rows(lst, .id = "Type")
dat = dat %>% tidyr::separate(Region, c("Chrom", "ChromStart", "ChromEnd"), remove = FALSE)
dat = dat %>% dplyr::select(Chrom, ChromStart, ChromEnd, Region, Type, Num_Assay)

dat_region_annot_fcc_vote = dat
print(dim(dat))
fun_display_table(head(dat, 3))

[1] 160666      6


,Chrom,ChromStart,ChromEnd,Region,Type,Num_Assay
chr1:115429-115969,chr1,115429,115969,chr1:115429-115969,Enhance,1
chr1:184091-184563,chr1,184091,184563,chr1:184091-184563,Enhance,1
chr1:605104-605675...3,chr1,605104,605675,chr1:605104-605675,Enhance,1


In [10]:
dat = dat_region_annot_fcc_vote
table(dat$Type)


          CRISPR          Enhance          Repress Repress_GCFilter 
            4740            43993            65401            46532 

In [11]:
dat = dat_region_annot_fcc_vote
table(dat$Type, dat$Num_Assay)

                  
                       1     2     3     4
  CRISPR            4692    40     8     0
  Enhance          27396 12548  4011    38
  Repress          63652  1744     5     0
  Repress_GCFilter 45310  1219     3     0

## Export results

In [12]:
txt_fdiry = file.path(
    FD_RES, 
    "region_annotation", 
    "fcc_astarr_macs_input_overlap",
    "summary"
)
txt_fname = "region.summary.fcc_peak_call.assayvote.tsv"
txt_fpath = file.path(txt_fdiry, txt_fname)

dat = dat_region_annot_fcc_vote
write_tsv(dat, txt_fpath)